In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The following document is from the open data of the Department of Conservation (DOC) in New Zealand. As seen below, it provides 3,262 unique trail names with global positioning

In [2]:
doc = pd.read_csv('data/DOC_Tracks.csv')
doc.head()
doc.shape
doc.info()

,OBJECTID,CAPTURE_ACCURACY,EQUIPMENT,DESCRIPTION,STATUS,OBJECT_TYPE_DESCRIPTION,GlobalID,SHAPE_Length
0,521495,2.0,100044818,TAUHERENIKAU GORGE TRACK,OPEN,Tramping Track,{3AAF81F3-73B3-4F88-A796-17B07DFF1A81},2310.732688
1,521496,2.0,100044819,MOUNT HOLDSWORTH TO JUMBO HUT,OPEN,Easy Tramping Track,{E84ABC5A-4EBE-4AE8-859E-48D9292A851E},3761.813276
2,521497,2.0,100044820,RAINGAUGE SPUR,OPEN,Easy Tramping Track,{83173518-A632-4A4D-8076-092A7851298F},2129.463229
3,521498,2.0,100044821,BARRA TRACK - THE PINES TO BLAKE STREAM,OPEN,Tramping Track,{96C9A611-59BA-4A5B-AD9B-ACAC7F3AC152},2046.447855
4,521499,2.0,100044822,EAST HOLDSWORTH TRACK,OPEN,Tramping Track,{E460CE11-526E-4DF0-99BF-9BF5EB00D38F},3508.240620


(3262, 8)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3262 entries, 0 to 3261
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   OBJECTID                 3262 non-null   int64  
 1   CAPTURE_ACCURACY         3260 non-null   float64
 2   EQUIPMENT                3262 non-null   int64  
 3   DESCRIPTION              3262 non-null   object 
 4   STATUS                   3262 non-null   object 
 5   OBJECT_TYPE_DESCRIPTION  3262 non-null   object 
 6   GlobalID                 3262 non-null   object 
 7   SHAPE_Length             3262 non-null   float64
dtypes: float64(2), int64(2), object(4)
memory usage: 204.0+ KB


Need some trail information for these tracks!

In [3]:
track_names = doc.DESCRIPTION.tolist()

In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

URL = 'https://walkingkiwi.co.nz/'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

D:\Downloads\Anaconda\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


<script>
 var trackArray = [];var trackLocation = {
        lng: 173.096578,
        lat: -41.494357,
        index: 0,
        time: '11h 39m',
        length: '11.51km',
        type: 'Tramping Track',
        url: '-l-wairoa-road-end-to-barrys-flat',
        name: ' L. Wairoa Road End to Barrys Flat'};
trackArray.push(trackLocation);var trackLocation = {
        lng: 176.006116,
        lat: -37.927966,
        index: 1,
        time: '2h 43m',
        length: '6.45km',
        type: 'Tramping Track',
        url: '-ngatuhoa-stream-track',
        name: ' Ngatuhoa Stream Track'};
trackArray.push(trackLocation);var trackLocation = {
        lng: 168.594405,
        lat: -43.975153,
        index: 2,
        time: '2h 18m',
        length: '5.84km',
        type: 'Route',
        url: '-coastal-smoothwaterstaffordhut-route',
        name: '(coastal) Smoothwater-staffordhut Route'};
trackArray.push(trackLocation);var trackLocation = {
        lng: 168.545144,
        lat: -43.996206,
 

In [5]:
script = soup.find('script')
script = str(script)
info = script.split(';')
# there are a few track names that had a semi-colon which needs to be corrected
info[287:289] = [''.join(info[287:289])]
info[289:291] = [''.join(info[289:291])]
info[2745:2747] = [''.join(info[2745:2747])]
info[5967:5969] = [''.join(info[5967:5969])]

In [6]:
split = []
for i in range(1,len(info),2):
    q = info[i].split('=')
    for n in range(1,len(q)):
        split.append(q[n])
        
k = []
for i in range(len(split)):
    k.append(split[i].replace('\n        ','').strip(' '))
    
data = []
for i in k:
    result = re.search('{(.*)}', i)
    each = result.group(1)
    data.append(each)


In [7]:
df = pd.DataFrame([p.split(',') for p in data], columns=['lng','lat','index','time','length','type','url','name','unknown'])

In [8]:
# unknown appeared due to commas in the name so we will fix that
df.unknown.fillna(value='', inplace=True)
df['name'] = df['name'] + '' + df['unknown']
df.drop(columns=['unknown','index'], inplace=True)
# remove string of column names from the data
df['lng'] = df['lng'].map(lambda x: x.lstrip('lng:'))
df['lat'] = df['lat'].map(lambda x: x.lstrip('lat:'))
df['time'] = df['time'].map(lambda x: x.lstrip("time: '").rstrip("'"))
df['length'] = df['length'].map(lambda x: x.lstrip("length: '").rstrip("km'"))
df.rename(columns={"length": "length_km"}, inplace=True)
df['type'] = df['type'].map(lambda x: x.lstrip("type: '").rstrip("'"))
df['url'] = df['url'].map(lambda x: x.lstrip("url: '").rstrip("'"))
df['name'] = df['name'].map(lambda x: x.lstrip("name: '").rstrip("'"))
# re-order the columns
df = df[['name', 'type', 'time', 'length_km', 'lng','lat','url']]
# adjust time from XXhXXm to hours
df['time'] = df['time'].map(lambda x: re.sub("[^0-9]", "", x))
import time
hours = []
for x in df.time:
    t = time.strptime(x,"%H%M")
    hours.append(t.tm_hour + (t.tm_min/60))
df['time']  = hours
df.rename(columns={"time": "time_h"}, inplace=True)

df.head()
df.shape
df.info()

,name,type,time_h,length_km,lng,lat,url
0,L. Wairoa Road End to Barrys Flat,Tramping Track,11.650000,11.51,173.096578,-41.494357,-l-wairoa-road-end-to-barrys-flat
1,Ngatuhoa Stream Track,Tramping Track,2.716667,6.45,176.006116,-37.927966,-ngatuhoa-stream-track
2,(coastal) Smoothwater-staffordhut Route,Route,21.133333,5.84,168.594405,-43.975153,-coastal-smoothwaterstaffordhut-route
3,(inland) Smoothwater-staffordhut Route,Route,2.983333,6.99,168.545144,-43.996206,-inland-smoothwaterstaffordhut-route
4,1000 Acres (1000 Ac Jcn to Pp Hut),Tramping Track,23.100000,4.42,172.331671,-41.649679,1000-acres-1000-ac-jcn-to-pp-hut


(3119, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3119 entries, 0 to 3118
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       3119 non-null   object 
 1   type       3119 non-null   object 
 2   time_h     3119 non-null   float64
 3   length_km  3119 non-null   object 
 4   lng        3119 non-null   object 
 5   lat        3119 non-null   object 
 6   url        3119 non-null   object 
dtypes: float64(1), object(6)
memory usage: 170.7+ KB


In [53]:
df.to_csv('data/WalkingKiwi_Tracks.csv')

In [9]:
# each hiking trail url to gain elevation information
track_urls = df.url.tolist()
print(track_urls)

['-l-wairoa-road-end-to-barrys-flat', '-ngatuhoa-stream-track', '-coastal-smoothwaterstaffordhut-route', '-inland-smoothwaterstaffordhut-route', '1000-acres-1000-ac-jcn-to-pp-hut', '1000-acres-pp-hut-to-lar-hut', '1220-track', '1st-carpark-easement', '3-mile-pack-track', '6-tk-jnc-near-te-panaa-to-taurawharona', '7mile-creek-bushline-to-dillons-hut-trk', 'abel-tasman-inland-trk-hoakesporters', 'abel-tasman-inland-trk-tlinehoakes', 'abel-tasman-memorial-walk', 'access-tracks-around-rangatira-flat', 'acheron-lookout-loop-walk', 'acland-falls-track', 'acton-bushedgeacton-cromel-route-junctn', 'acton-hut-sitecromel-base-hut-junctn', 'acton-cromel-rte-junctcromelbranch-hut', 'ada-track', 'adeanes-bush-sr-track', 'adventure-ridge-route', 'ahukawakawa-track', 'ahuriri-estuary-track', 'ahuriri-valley-track', 'ake-ake-pt-track', 'akersten-bay-track', 'albert-burn-track', 'alborns-walk', 'alder-track', 'alex-knob-track', 'alexandra-210-to-205', 'alford-forest-easement', 'alice-nash-h-lodge-to-ir

In [11]:
# start them empty over here and then we'll add to them as we parse through the urls
coordinates = []
trackElevation = []
length_confirm = []
maxElevation = []
minElevation = []

In [74]:
def scrape_hikes(track_urls):
    for url in track_urls:
        URL = 'https://walkingkiwi.co.nz/track/' + str(url)
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
    
        script = soup.find_all('script')[1]
        script = str(script)
        info = script.split(';')
    
        coordinates.append(info[2])
        trackElevation.append(info[3])
        length_confirm.append(info[4])
        maxElevation.append(info[5])
        minElevation.append(info[6])
        soup.decompose()
    print('Scrape Complete. Lists Appended')

In [83]:
df.iloc[91]

name         Atc - Tinline Nature Walk
type                     Walking Track
time_h                         1.11667
length_km                         0.65
lng                         173.023830
lat                         -40.987142
url            atc-tinline-nature-walk
Name: 91, dtype: object

In [82]:
length_confirm[-1]

' var trackLength = 0.65'

In [81]:
len(length_confirm)

92

In [84]:
scrape_hikes(track_urls[92:100])

Scrape Complete. Lists Appended


In [ ]:
df['coordinates'] = coordinates
df['trackElevation'] = trackElevation
df['lengthcheck'] = length_confirm
df['maxElevation'] = maxElevation
df['minElevation'] = minElevation
df.head()